In [2]:
import sys
import sqlite3, csv
from datetime import datetime as dt
import datetime

In [3]:
%load_ext sql

C:\Users\mtags\Anaconda3\lib\site-packages\IPython\config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
C:\Users\mtags\Anaconda3\lib\site-packages\IPython\utils\traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [ ]:
%%sql sqlite:///animal_tracker.db
DROP TABLE IF EXISTS animal_tracker_raw;

CREATE TABLE animal_tracker_raw ( 
    actual_lot_size INTEGER, 
    lot_tag_read_count INTEGER, 
    departure_cph TEXT, 
    destination_cph TEXT, 
    read_location_cph TEXT, 
    lot_date DATE,
    timestamp DATE,
    movement_type INTEGER, 
    animal_eid INTEGER, 
    flock_tag INTEGER, 
    tag_count INTEGER, 
    issue_cph TEXT, 
    issue_date DATE, 
    last_update DATE, 
    unique_lot_id INTEGER) 

In [ ]:
%%sql sqlite:///animal_tracker.db
    
CREATE INDEX IF NOT EXISTS `animal_eid_idx` ON `animal_tracker_raw` (`animal_eid` ASC);
CREATE INDEX IF NOT EXISTS `lot_date_idx` ON `animal_tracker_raw` (`lot_date` ASC);
CREATE INDEX IF NOT EXISTS `timestamp_idx` ON `animal_tracker_raw` (`timestamp` ASC);
CREATE INDEX IF NOT EXISTS `departure_cph_idx` ON `animal_tracker_raw` (`departure_cph` ASC);
CREATE INDEX IF NOT EXISTS `destination_cph_idx` ON `animal_tracker_raw` (`destination_cph` ASC);
CREATE INDEX IF NOT EXISTS `read_location_cph_idx` ON `animal_tracker_raw` (`read_location_cph` ASC);



# Import the data from csv file
the cell below imports the raw data csv file into the **animal_tracker_raw** table.

change the path & filename of the csv file as needed

In [ ]:
import sys
import sqlite3, csv
from datetime import datetime as dt
import datetime
import time
import traceback

def cleanDate(date):
    if date != '\\N':
        return dt.strptime(date, '%Y-%m-%d')
    else:
        return dt.strptime('1900-01-01', '%Y-%m-%d')

conn = sqlite3.connect("animal_tracker.db")

try:
    start = time.time()

    conn.execute("delete from animal_tracker_raw");

    cur = conn.cursor()
    row_num = 0
    with open('out_useful_only_ts.csv') as openFile:
        #dr = csv.DictReader(openFile)
        dr = csv.reader(openFile)   
        firstRow = True
        for i in dr:
            if not firstRow:
                row_num += 1
                if row_num % 100000 == 0:
                    print(row_num)

                cur.execute("""
                    INSERT INTO animal_tracker_raw (
                    actual_lot_size,
                    lot_tag_read_count,
                    departure_cph,
                    destination_cph,
                    read_location_cph,
                    lot_date,
                    timestamp,
                    movement_type,
                    animal_eid,
                    flock_tag,
                    tag_count,
                    issue_cph,
                    issue_date,
                    last_update,
                    unique_lot_id 
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);

                    """, i)
            firstRow = False
        
        
    conn.commit()
    print('done in db ' + str(time.time() - start))
except:
    print("Unexpected error:", sys.exc_info()[0])
    traceback.print_exc()
else:
    conn.close()

print("done")

### data columns 
actual_lot_size - the size of the lot that the animal belongs to (a group as they go to a desitination)?
lot_tag_read_count - the count of animals, within the lot, whose tag was read at the read_location_cph?
departure_cph - where the animal was before the reading location
destination_cph - the location where the animal is going next (a farm, a slaughter house, a shipping port, may leave the system)? after the read location
read_location_cph - the location where the animals tag was read for this record (often at a livestock or similar 3rd party)?
lot_date - the date the tag was read for this animal for this record?
movement_type - some detail about the departure and desitination - farm to farm, farm to slaughter, farm to port? This help identify that the animal has left the system (because it was killed at a slaughter)
animal_eid - the unique identifier for an animal?
flock_tag - a single tag of a full flock/lot - not individual sheep
tag_count - associated with flock tag, the number of animals scanned within the flock with the same flock tag
issue_cph - birth farm of animal, where the tag was issued to the animal
issue_date - date the animal got the tag for the issue_cph
last_update - ???
unique_lot_id - exactly

cph (county/perish/holding)


patient zero tracking
tracking window is to see when the disese started and likely ended
the slower the disease moves, the larger the window

network analysis
degrees of fredom


the result is all the sheep that the animal of interest was in contact with
    possibley of showing a weight of how long they were in contact
    show other animals current location (last scan)
    show on a pretty graph that is easy to understand
    
Also need to identify when a sheep disappeared from the system (due to lack of scanning)





In [ ]:
%%sql sqlite:///animal_tracker.db

select * from animal_tracker_raw limit 100;

In [ ]:
%%sql sqlite:///animal_tracker.db

select animal_eid, lot_date, count(lot_date) as inst from animal_tracker_raw Group By animal_eid, lot_date having inst > 2;

In [ ]:
%%sql sqlite:///animal_tracker.db

select animal_eid, lot_date, departure_cph, destination, read_location_cph from animal_tracker_raw where animal_eid = 105908173 order by lot_date;

In [ ]:
%%sql sqlite:///animal_tracker.db
select data_record.animal_eid, data_record.lot_date, data_record.timestamp, data_record.destination_cph, case when timestamp != '\N' then date(timestamp) else date(lot_date) end as movement_date from animal_tracker_raw as data_record
left join (
select animal_eid, theday, max(thedatetime) maxdatetime from (
select animal_eid, case when timestamp != '\N' then date(timestamp) else date(lot_date) end as theday, case when timestamp != '\N' then timestamp else lot_date end as thedatetime  from animal_tracker_raw where animal_eid = 10185700272 limit 1000
    ) as max_date_record group by animal_eid, theday
) as last_record_of_day where last_record_of_day.animal_eid = data_record.animal_eid AND (last_record_of_day.maxdatetime = data_record.lot_date OR last_record_of_day.maxdatetime = data_record.timestamp)

In [ ]:
%%sql sqlite:///animal_tracker.db

select min(movement_date) from (
select case when timestamp != '\N' then timestamp else lot_date end as movement_date  from animal_tracker_raw
    )


In [ ]:
%%sql sqlite:///animal_tracker.db
select count(*) from (select distinct animal_eid, unique_lot_id from animal_tracker_raw);

In [ ]:
%%sql sqlite:///animal_tracker.db
select departure_cph, destination_cph, read_location_cph, lot_date, timestamp, animal_eid from animal_tracker_raw where animal_eid = 10185700272  order by timestamp

In [40]:
%%sql sqlite:///animal_tracker.db
select animal_eid, count(animal_eid) as c from  animal_tracker_raw group by animal_eid having c > 5;

Done.


animal_eid,c
1,9
227,8
520874,7
542610,6
542859,6
544971,6
545107,6
560312,10
563244,6
105908173,6


In [ ]:
%%sql sqlite:///animal_tracker.db
select * from animal_location;

In [ ]:
%%sql sqlite:///animal_tracker.db
update animal_location set location_cph = "08/188/0105" where animal_eid = 10185700272 and location_cph = "08/124/0026";


# sheep location duration
below is made-up data as an example of how the raw data should be transformed (TBD)

In [ ]:
%%sql sqlite:///animal_tracker.db
DROP TABLE IF EXISTS animal_location;

CREATE TABLE animal_location(animal_eid INTEGER, location_cph VARCHAR(16), arrival_date DATE, departure_date DATE, location_type VARCHAR(10));

CREATE INDEX IF NOT EXISTS `animal_location_animal_eid_idx` ON `animal_location` (`animal_eid` ASC);
CREATE INDEX IF NOT EXISTS `animal_location_location_cph_idx` ON `animal_location` (`location_cph` ASC);
CREATE INDEX IF NOT EXISTS `animal_location_arrival_date_idx` ON `animal_location` (`arrival_date` ASC);
CREATE INDEX IF NOT EXISTS `animal_location_departure_date_idx` ON `animal_location` (`departure_date` ASC);


In [5]:
import sys
import sqlite3, csv
import copy
from datetime import datetime as dt
import datetime
import traceback
sys.path.append('C:\\Python27\\Lib\\site-packages\\MySQLdb')

dbpath = "animal_tracker.db"

nowDateTime = datetime.datetime.now()

def dosomething(firstDateTime):
    arrival_date = firstDateTime
    departure_date = firstDateTime
    previous_animal_id = 0
    previous_destination = ""
    
    results_animal_tracker_raw = conn.execute(" \
        select data_record.animal_eid, data_record.lot_date, data_record.timestamp, data_record.destination_cph, data_record.destination_cph, data_record.departure_cph, data_record.read_location_cph, \
        case when timestamp != '\\N' then date(timestamp) else date(lot_date) end as movement_date \
        from animal_tracker_raw as data_record  \
        left join ( \
        select animal_eid, theday, max(thedatetime) maxdatetime from ( \
            select animal_eid, case when timestamp != '\\N' then date(timestamp) else date(lot_date) end as theday, \
            case when timestamp != '\\N' then timestamp else lot_date end as thedatetime \
            from animal_tracker_raw where animal_eid IN(10185700272, 1, 10763600332) limit 1000 \
            ) as max_date_record group by animal_eid, theday \
        ) as last_record_of_day where last_record_of_day.animal_eid = data_record.animal_eid AND (last_record_of_day.maxdatetime = data_record.lot_date OR last_record_of_day.maxdatetime = data_record.timestamp) \
        order by data_record.animal_eid, movement_date", )
    for row_animal_tracker_raw in results_animal_tracker_raw:
        
        departure_date = row_animal_tracker_raw['movement_date'] + " 00:00:00.000"

        if previous_animal_id == row_animal_tracker_raw['animal_eid'] :
            
            #departure
            print(str(previous_animal_id) + " " + previous_destination + " " + str(arrival_date) + " " + str(departure_date) + " dest")
            insertIntoAnimalLocation(previous_animal_id, previous_destination, arrival_date, departure_date, "dest")
        
            arrival_date = departure_date
        else: # if the animal id has changed - this will be the last record of the previous animal id
            
            #departure
            if previous_animal_id != 0:
                print(str(previous_animal_id) + " " + previous_destination + " " + str(arrival_date) + " " + str(nowDateTime) + " dest last")
                insertIntoAnimalLocation(previous_animal_id, previous_destination, arrival_date, nowDateTime, "dest last")
            
            arrival_date = firstDateTime # set the arrival date for the next animal
        
        departure_date = row_animal_tracker_raw['movement_date'] + " 01:00:00.000"
        print(str(row_animal_tracker_raw['animal_eid']) + " " + row_animal_tracker_raw['departure_cph'] + " " + str(arrival_date) + " " + str(departure_date) + " dep")
        insertIntoAnimalLocation(row_animal_tracker_raw['animal_eid'], row_animal_tracker_raw['departure_cph'], arrival_date, departure_date, "dep")
        arrival_date = departure_date
        departure_date = row_animal_tracker_raw['movement_date'] + " 02:00:00.000"
        print(str(row_animal_tracker_raw['animal_eid']) + " " + row_animal_tracker_raw['read_location_cph'] + " " + str(arrival_date) + " " + str(departure_date) + " read")
        insertIntoAnimalLocation(row_animal_tracker_raw['animal_eid'], row_animal_tracker_raw['read_location_cph'], arrival_date, departure_date, "read")
        arrival_date = departure_date
       
        previous_animal_id = row_animal_tracker_raw['animal_eid']
        previous_destination = row_animal_tracker_raw['destination_cph']
    
    print(str(previous_animal_id) + " " + previous_destination + " " + str(arrival_date) + " " + str(nowDateTime) + " dest last")
    insertIntoAnimalLocation(previous_animal_id, previous_destination, arrival_date, nowDateTime, "dest last")

def getFirstDate():
    min_timestamp = dt.today()
    results_mints = conn.execute( \
                "SELECT min(timestamp) as min_timestamp from animal_tracker_raw \
                WHERE timestamp > '2000-01-01'", \
                )
    for row_mints in results_mints:
        min_timestamp = row_mints['min_timestamp']
    
    return min_timestamp

def insertIntoAnimalLocation(animal_eid, location, arrival_date, departure_date, location_type):
    conn.execute("INSERT INTO animal_location VALUES (?, ?, ?, ?, ?)", \
                    (animal_eid, location, arrival_date, departure_date, location_type));


conn = sqlite3.connect(dbpath)
conn.row_factory = sqlite3.Row
try:
    conn.execute("delete from animal_location");
    

    dosomething(getFirstDate())

except:
    print("Unexpected error:", sys.exc_info()[0])
    traceback.print_exc()
else:
    conn.commit()
    conn.close()



1 75/309/0043 2007-01-17 16:06:32 2011-02-21 01:00:00.000 dep
1 66/083/8000 2011-02-21 01:00:00.000 2011-02-21 02:00:00.000 read
1 66/083/8000 2011-02-21 02:00:00.000 2011-02-24 00:00:00.000 dest
1 08/181/0041 2011-02-24 00:00:00.000 2011-02-24 01:00:00.000 dep
1 08/181/8000 2011-02-24 01:00:00.000 2011-02-24 02:00:00.000 read
1 49/521/8000 2011-02-24 02:00:00.000 2011-04-04 00:00:00.000 dest
1 66/074/0023 2011-04-04 00:00:00.000 2011-04-04 01:00:00.000 dep
1 66/083/8000 2011-04-04 01:00:00.000 2011-04-04 02:00:00.000 read
1 66/083/8000 2011-04-04 02:00:00.000 2011-04-14 00:00:00.000 dest
1 08/181/0041 2011-04-14 00:00:00.000 2011-04-14 01:00:00.000 dep
1 08/181/8000 2011-04-14 01:00:00.000 2011-04-14 02:00:00.000 read
1 29/184/0066 2011-04-14 02:00:00.000 2011-04-28 00:00:00.000 dest
1 08/181/0041 2011-04-28 00:00:00.000 2011-04-28 01:00:00.000 dep
1 08/181/8000 2011-04-28 01:00:00.000 2011-04-28 02:00:00.000 read
1 08/188/0105 2011-04-28 02:00:00.000 2011-05-05 00:00:00.000 dest
1 08

In [ ]:
%%sql sqlite:///animal_tracker.db
select * from animal_location where location_cph = "08/181/8000";

In [4]:
%%sql sqlite:///animal_tracker.db
DELETE FROM animal_location;
INSERT INTO animal_location VALUES ("s1", "l1", "2016-01-01 12:00:00.000", "2016-02-01 00:00:00.000", "test");
INSERT INTO animal_location VALUES ("s1", "l2", "2016-02-01 12:00:00.000", "2016-05-01 00:00:00.000", "test");

INSERT INTO animal_location VALUES ("s2", "l2", "2016-01-01 12:00:00.000", "2016-03-01 00:00:00.000", "test");
INSERT INTO animal_location VALUES ("s2", "l3", "2016-03-01 12:00:00.000", "2016-05-01 00:00:00.000", "test");

INSERT INTO animal_location VALUES ("s3", "l3", "2016-01-01 12:00:00.000", "2016-04-01 00:00:00.000", "test");
INSERT INTO animal_location VALUES ("s3", "l4", "2016-04-01 12:00:00.000", "2016-05-01 00:00:00.000", "test");

INSERT INTO animal_location VALUES ("s4", "l4", "2016-01-01 12:00:00.000", "2016-05-01 00:00:00.000", "test");

INSERT INTO animal_location VALUES ("s5", "l4", "2016-01-01 12:00:00.000", "2016-02-01 00:00:00.000", "test");
INSERT INTO animal_location VALUES ("s5", "l1", "2016-02-01 12:00:00.000", "2016-05-01 00:00:00.000", "test");



102 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [7]:
%%sql sqlite:///animal_tracker.db
DELETE FROM animal_location;
INSERT INTO animal_location VALUES ("1", "l1", "2016-01-01 12:00:00.000", "2016-02-01 00:00:00.000", "test");
INSERT INTO animal_location VALUES ("1", "l2", "2016-02-01 12:00:00.000", "2016-05-01 00:00:00.000", "test");

INSERT INTO animal_location VALUES ("s2", "l1", "2016-01-01 12:00:00.000", "2016-02-01 00:00:00.000", "test");
INSERT INTO animal_location VALUES ("s2", "l2", "2016-02-01 12:00:00.000", "2016-05-01 00:00:00.000", "test");

INSERT INTO animal_location VALUES ("s3", "l2", "2016-01-01 12:00:00.000", "2016-05-01 00:00:00.000", "test");

INSERT INTO animal_location VALUES ("s4", "l2", "2016-01-01 12:00:00.000", "2016-05-01 00:00:00.000", "test");

INSERT INTO animal_location VALUES ("s5", "l4", "2016-01-01 12:00:00.000", "2016-05-01 00:00:00.000", "test");


7 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [ ]:
%%sql sqlite:///animal_tracker.db
SELECT * FROM animal_location;

# end sheep location duration

## algorithm to find affected sheep

In [ ]:
%%sql sqlite:///animal_tracker.db
DROP TABLE IF EXISTS exposure_incident;
CREATE TABLE exposure_incident ( 
    giver_animal_eid INTEGER, 
    location_cph VARCHAR(16), 
    location_exposure_date TIMESTAMP, 
    receiver_animal_eid INTEGER, 
    animal_exposure_date TIMESTAMP 
);


In [43]:
import sys
import sqlite3, csv
import copy
from datetime import datetime as dt
import datetime
import traceback

dbpath = "animal_tracker.db"

insertCount = 0

def process_exposures(insertCount):
    isNewRecordsCreated = False
    results_ei_giver = conn.execute("SELECT * FROM exposure_incident")            
    for row_giver in results_ei_giver.fetchall():
        #print(row_giver['receiver_animal_eid'] + "\t" + row_giver['animal_exposure_date'])

        results_ei_receiver = conn.execute(" \
            select locq.giver_animal_eid, locq.location_cph, locq.location_exposure_date, shpq.animal_eid as receiver_animal_eid, \
                CASE WHEN arrival_date > locq.location_exposure_date then arrival_date else locq.location_exposure_date end AS animal_exposure_date from animal_location shpq, \
            (select animal_eid as giver_animal_eid, location_cph, min(CASE WHEN arrival_date > ? then arrival_date else ? end) AS location_exposure_date \
            from animal_location where animal_eid = ? and departure_date >= ? GROUP BY animal_eid, location_cph) as locq \
            where shpq.location_cph = locq.location_cph and shpq.departure_date > locq.location_exposure_date", \
            (row_giver['animal_exposure_date'], row_giver['animal_exposure_date'], row_giver['receiver_animal_eid'], row_giver['animal_exposure_date'], ))
        for row_receiver in results_ei_receiver:
            #print(row_receiver[0] + "\t" + row_receiver[1] + "\t" + row_receiver[2] + "\t" + row_receiver[3] + "\t" + row_receiver[4])
            
            doInsertRecord = True
            cursor = conn.cursor()
            #check that the giver and receiver animal arent the same
            if row_receiver['giver_animal_eid'] == row_receiver['receiver_animal_eid']:
                doInsertRecord = False

            #check that you arent exposing an animal that exposed you
            if doInsertRecord:
                cursor.execute( \
                    "SELECT giver_animal_eid FROM exposure_incident \
                    WHERE giver_animal_eid = ? AND receiver_animal_eid = ? LIMIT 1", \
                    (row_receiver['receiver_animal_eid'], row_receiver['giver_animal_eid'],))
                if cursor.fetchone() is not None:
                    doInsertRecord = False

            #check to ensure record doesnt exist already
            if doInsertRecord:
                cursor.execute( \
                    "SELECT giver_animal_eid FROM exposure_incident \
                    WHERE giver_animal_eid = ? AND location_cph = ? AND location_exposure_date = ? AND receiver_animal_eid = ? AND animal_exposure_date = ? LIMIT 1", \
                    (row_receiver['giver_animal_eid'], row_receiver['location_cph'], row_receiver['location_exposure_date'], row_receiver['receiver_animal_eid'], row_receiver['animal_exposure_date'],))
                if cursor.fetchone() is not None:
                    doInsertRecord = False

            if doInsertRecord:
                conn.execute("INSERT INTO exposure_incident (giver_animal_eid, location_cph, location_exposure_date, receiver_animal_eid, animal_exposure_date) VALUES (?, ?, ?, ?, ?)", \
                    (row_receiver['giver_animal_eid'], row_receiver['location_cph'], row_receiver['location_exposure_date'], row_receiver['receiver_animal_eid'], row_receiver['animal_exposure_date'],))
                
                print("%s %s %s %s %s %s" % (row_receiver['giver_animal_eid'], row_receiver['location_cph'], row_receiver['location_exposure_date'], row_receiver['receiver_animal_eid'], row_receiver['animal_exposure_date'], insertCount))
                insertCount += 1
                
                isNewRecordsCreated = True
            cursor.close()
    if isNewRecordsCreated:
        #print("new records created " + str(datetime.datetime.now()))
        process_exposures(insertCount)
        isNewRecordsCreated = False
    return 


conn = sqlite3.connect(dbpath)
conn.row_factory = sqlite3.Row
try:

    conn.execute("DELETE FROM exposure_incident")
    conn.execute("INSERT INTO exposure_incident (receiver_animal_eid, animal_exposure_date) VALUES (?, ?)", ('1', '2006-02-01 00:00:00.000',))

    process_exposures(insertCount)
    
    print("giver_animal_eid" + "\t" + "location_cph" + "\t" + "location_exposure_date" + "\t" + "receiver_animal_eid" + "\t" + "animal_exposure_date")
    results_ei_final = conn.execute("SELECT * FROM exposure_incident")            
    for row in results_ei_final:
        print(str(row['giver_animal_eid']) + "\t" + str(row['location_cph']) + "\t" + str(row['location_exposure_date']) + "\t" + str(row['receiver_animal_eid']) + "\t" + str(row['animal_exposure_date']))
    
except:
    print("Unexpected error:", sys.exc_info()[0])
    traceback.print_exc()
else:
    conn.commit() 
    conn.close()


new records created 2016-10-11 10:34:22.308913
new records created 2016-10-11 10:34:22.311413
new records created 2016-10-11 10:34:22.313413
new records created 2016-10-11 10:34:22.315413
new records created 2016-10-11 10:34:22.317913
new records created 2016-10-11 10:34:22.320914
new records created 2016-10-11 10:34:22.325415
new records created 2016-10-11 10:34:22.328415
new records created 2016-10-11 10:34:22.331416
new records created 2016-10-11 10:34:22.335416
new records created 2016-10-11 10:34:22.338917
new records created 2016-10-11 10:34:22.343918
new records created 2016-10-11 10:34:22.349918
new records created 2016-10-11 10:34:22.360419
new records created 2016-10-11 10:34:22.366920
new records created 2016-10-11 10:34:22.371440
new records created 2016-10-11 10:34:22.375440
new records created 2016-10-11 10:34:22.379941
new records created 2016-10-11 10:34:22.384942
new records created 2016-10-11 10:34:22.392943
new records created 2016-10-11 10:34:22.400943
new records c

In [ ]:
%%sql sqlite:///animal_tracker.db
select * from exposure_incident;

In [ ]:
%matplotlib inline

import sys
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime as dt


conn = sqlite3.connect("animal_tracker.db")
#conn.row_factory = sqlite3.Row
#giver_animal_eid	location_cph	location_exposure_date	receiver_animal_eid	animal_exposure_date


results_max_departure_date = conn.execute("SELECT max(departure_date) FROM animal_location")
for row_max_departure_date in results_max_departure_date:
    max_departure_date = mdates.date2num(dt.strptime(row_max_departure_date[0], '%Y-%m-%d %H:%M:%S.%f')) 
print(max_departure_date)

location_names_values = {}

## find the locations
results_loc_exp = conn.execute("SELECT location_cph, min(location_exposure_date) FROM exposure_incident WHERE location_cph != 'None' GROUP BY location_cph ORDER BY location_exposure_date")
loc_labels = []
loc_rows = []
loc_startdates = []
loc_left = []
loc_enddates = []
loc_row_count = 0
for row_loc_exp in results_loc_exp:
    loc_labels.append(row_loc_exp[0])
    loc_row_count += 1
    location_names_values[row_loc_exp[0]] = loc_row_count
    loc_rows.append(loc_row_count)
    loc_startdate = mdates.date2num(dt.strptime(row_loc_exp[1], '%Y-%m-%d %H:%M:%S.%f'))
    loc_startdates.append(loc_startdate)
        
    loc_enddates.append(max_departure_date - loc_startdate)
##end find the locations


## find the animals
results_animal_exp = conn.execute("SELECT giver_animal_eid, location_cph, receiver_animal_eid, animal_exposure_date FROM exposure_incident WHERE location_cph != 'None' ORDER BY animal_exposure_date")
giver_animal_eids = []
receiver_animal_eids = []
animal_exposure_dates = []
animal_exposure_locs = []

exposure_labels = {}
for row_animal_exp in results_animal_exp:
    animal_exposure_date = mdates.date2num(dt.strptime(row_animal_exp[3], '%Y-%m-%d %H:%M:%S.%f'))
    animal_exposure_dates.append(animal_exposure_date)
    animal_exposure_locs.append(location_names_values[row_animal_exp[1]])
    exposure_labels[str(row_animal_exp[0]) + "->" + str(row_animal_exp[2])] = (animal_exposure_date, location_names_values[row_animal_exp[1]])
    #ax.annotate(row_animal_exp[0] + "->" + row_animal_exp[2], xy=(animal_exposure_date, location_names_values[row_animal_exp[1]]), textcoords='data')

##end find the animals
print(exposure_labels)
    
conn.close()

print(location_names_values)

print(loc_labels)
print(loc_rows)
print(loc_startdates)
print(loc_enddates)


fig, ax = plt.subplots()


ax.plot(animal_exposure_dates, animal_exposure_locs, 'ro')
ax.barh(loc_rows, loc_enddates, left=loc_startdates )

for label in exposure_labels:
    ax.annotate(label, xy=exposure_labels[label], textcoords='data')

plt.yticks(loc_rows, loc_labels)

ax.axis('tight')
ax.xaxis_date()
fig.autofmt_xdate()

plt.show()

# end algorithm to find affected sheep

# Below are cells that arent needed right now

### Research
Diffusion Model - Set of mathematical equations or formulas that attempts to estimate the spread of information (idea or rumor) or a contagious disease through a population.



Below:
Enter the date for time_of_interest yyyy mm dd
This will query the data and return the records where the issue_date is between the time_of_interest and the time_of_interest + 28 days
It then randomly selects one of the returned records

In [ ]:
from random import randint

time_of_interest = datetime.date(2011, 9, 1)
time_of_interest_end = time_of_interest + datetime.timedelta(days=28)

conn = sqlite3.connect(dbpath)
#
c1 = conn.execute("SELECT * FROM sheep_tracker WHERE lot_date BETWEEN ? AND ?", (time_of_interest, time_of_interest_end,))

all_rows = c1.fetchall()
rows_count = len(all_rows)
if rows_count > 0:
    random_row = all_rows[randint(0, rows_count - 1)]
    print(random_row)
    
    if random_row[5] < : 
    
    
else:
    print("no records found")



conn.close()

In [ ]:
from datetime import datetime as dt
import datetime
newd = dt.strptime('8/14/2010', '%m/%d/%Y')
time_of_interest = datetime.date(2011, 9, 1)
print(newd)
print(time_of_interest)

In [ ]:
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db
#%sql delete from mydata
%sql select animal_eid, count(*) from sheep_tracker GROUP BY animal_eid

## questions
1.  when selecting an animal within the time of interest, you want the records where the time_of_interest date range (TOI_start, TOI_end) overlaps with the animals time at a lot (lot_date, lot_date_end(need to find this based on next record) )  WHERE (lot_start < TIO_end AND lot_end >= TIO_start
2.  maybe need more data as each animal is only in the data once.  Or I can make some up.
3.  what will be the format of the outputs - fancy graphs or just data


# Chris's algorithm

1.  process raw data to show a sheepid, location, arrival date, departure date
    * this is where some data can be fudged to estimate lost data
2.  by selecting a sheep and a date range (disregarding the data outside of the range), determine which other sheep the primary sheep interacted with by matching locations and date overlaps
3.  (optional) apply a weight to how closely related the other sheep were to the primary sheep
    *  if sheep2 spent the whole date range with the primary sheep, then 1
    *  if only spent half the time then 0.5
    *  for each degree of separation the weight is further divided by 2 (* 0.5)
4.  show in fancy chart

In [ ]:
#this was just used to fine tune my query to find exposure_incident
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db

%sql select locq.*, shpq.animal_eid as animal_eid_receiver, CASE WHEN arrival_date > locq.loc_exposure_date then arrival_date else locq.loc_exposure_date end AS animal_exposure_date from sheep_location shpq, \
    (select animal_eid as animal_eid_giver, location_cph, min(CASE WHEN arrival_date > '2016-02-01' then arrival_date else '2016-02-01' end) AS loc_exposure_date \
    from sheep_location where animal_eid = 's1' and departure_date >= '2016-02-01' GROUP BY animal_eid, location_cph) as locq \
    where shpq.location_cph = locq.location_cph and shpq.departure_date > locq.loc_exposure_date;

# %sql select *, location_cph, min(CASE WHEN arrival_date > '2016-02-01' then arrival_date else '2016-02-01' end) AS exposure_date \
#     from sheep_location where animal_eid = 's1' and departure_date >= '2016-02-01' ;


In [ ]:
%sql sqlite:///C:\\Users\\mtags\\Desktop\\sheep.db

        
# %sql select * from sheep_location


%sql  select locq.giver_animal_eid, locq.location_cph, locq.location_exposure_date, shpq.animal_eid as receiver_animal_eid, CASE WHEN arrival_date > locq.location_exposure_date then arrival_date else locq.location_exposure_date end AS animal_exposure_date from sheep_location shpq, \
            (select animal_eid as giver_animal_eid, location_cph, min(CASE WHEN arrival_date > '2016-04-01 12:00:00.000' then arrival_date else '2016-04-01 12:00:00.000' end) AS location_exposure_date \
            from sheep_location where animal_eid = 's4' and departure_date >= '2016-04-01 12:00:00.000' GROUP BY animal_eid, location_cph) as locq \
            where shpq.location_cph = locq.location_cph and shpq.departure_date > locq.location_exposure_date
        
# %sql select animal_eid as giver_animal_eid, location_cph, min(CASE WHEN arrival_date > '2016-02-01 00:00:00.000' then arrival_date else '2016-02-01 00:00:00.000' end) AS location_exposure_date \
#             from sheep_location where animal_eid = 's1' and departure_date >= '2016-02-01 00:00:00.000' GROUP BY animal_eid, location_cph
    
# %sql select * from sheep_location \
#     where departure_date >= '2016-02-01 00:00:00.000'

In [ ]:
#I think this is an old algorithm but want to hand onto it just in case 2016-09-21 14:46

def dosomething(conn, infected_sheeps):
    print("entering dosomething")
    new_rec_found = False
    infected_loc_dict = {}
    # 1. identify the infected sheep
    for infected_sheep in infected_sheeps:
        # 2. store all the locations that the infected sheep(s) was at and the date range that the location was infected
        
        results = conn.execute( \
            "SELECT location_cph, min(arrival_date) FROM sheep_location WHERE animal_eid = ? AND departure_date >= ? GROUP BY location_cph", \
                (infected_sheep, infected_sheeps[infected_sheep]))
        for row in results:
            infected_loc_dict[row[0]] = row[1];
            

    print(infected_loc_dict)    
    #infected_sheeps = {};
# 3. search all sheep for the infected location/time range
    for loc in infected_loc_dict:
        #print(loc + " " + infected_loc_dict[loc])
        resultsR2 = conn.execute(\
            "SELECT animal_eid, min(arrival_date) FROM sheep_location WHERE location_cph = ? AND departure_date >= ? GROUP BY animal_eid", \
                (loc, infected_loc_dict[loc],))
        
        #print('aaa' + str(infected_sheeps))
        for rowR2 in resultsR2:
            print('ddd' + str(rowR2))
            if rowR2[0] not in infected_sheeps or infected_sheeps[rowR2[0]] > rowR2[1]:
                #print(str(rowR2[0] not in infected_sheeps))                
                #if rowR2[0] in infected_sheeps:
                    #print('\t' + str(infected_sheeps[rowR2[0]] > rowR2[1]))
                
                infected_sheeps[rowR2[0]] = rowR2[1]                
                new_rec_found = True
    if new_rec_found:
        print(infected_sheeps)
        infected_sheeps = dosomething(conn, infected_sheeps)
            
    return infected_sheeps



connX = sqlite3.connect(dbpath)

#try:
#d = datetime.date(2016, 1, 1)
s = '2016-04-01'
print(d)
infected_sheeps = {'s1': s}

infected_sheeps_result = dosomething(connX, infected_sheeps)
print(infected_sheeps_result)
#except:
print("Unexpected error:", sys.exc_info()[0])
#else:
connX.close()


In [ ]:
from IPython.core.display import display, HTML
display(HTML('<span style="color:red;">Hello, world!</span>'))

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.plot([2, 3, 2], [4, 4, 2], 'r^:')
plt.plot([1, 2, 3], [1, 2, 3], 'gs-')
plt.axis([0, 5, 0, 10])
plt.ylabel('some numbers')
plt.show()